In [1]:
import pandas as pd
import numpy as np
import requests
import aiohttp
import asyncio
import time


API_KEY = ""

In [2]:
df = pd.read_csv("temperature_data.csv")
df.head()

,city,timestamp,temperature,season
0,New York,2010-01-01,3.225691,winter
1,New York,2010-01-02,-1.491755,winter
2,New York,2010-01-03,-5.728947,winter
3,New York,2010-01-04,3.179504,winter
4,New York,2010-01-05,1.976765,winter


Сделаем в синхронном\асинхронном варианте функции получения текущей температуры по городу, функцию проверки температуры на нормальность, а также сделаем а\синхронную обертку для этих функций.

In [3]:
def curr_temp(city):
    response = requests.get(f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric")
    if response.status_code == 200:
        answer = response.json()
        return answer['main']['temp']
    else:
        raise Exception(f"Не удалось получить данные для города {city}. Код ответа - {response.status_code}")
    
# curr_temp("New York")

In [4]:
async def curr_temp_async(city):
    async with aiohttp.ClientSession() as session:
        async with session.get(f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric") as response:
            if response.status == 200:
                answer = await response.json()
                return answer['main']['temp']
            else:
                raise Exception(f"Не удалось получить данные для города {city}. Код ответа - {response.status}")
            

# await curr_temp_async("New York")

In [5]:
def is_temperature_normal(city, season, temp, df_grouped):
    stats = df_grouped[(df_grouped['city'] == city) & (df_grouped['season'] == season)]
    mean = stats['mean'].values[0]
    std = stats['std'].values[0]

    return mean - 2 * std <= temp <= mean + 2 * std

# grouped_df = df.groupby(['city', 'season'])['temperature'].agg(['mean', 'std']).reset_index()
# is_temperature_normal("New York", "winter", curr_temp("New York"), grouped_df)

In [6]:
def check_temperature_sync(city, season, df_grouped):
    cur_temp = curr_temp(city)
    is_normal = is_temperature_normal(city, season, cur_temp, df_grouped)
    return cur_temp, is_normal

# check_temperature_sync("New York", "winter", grouped_df)

In [7]:
async def check_temperature_async(city, season, df_grouped):
    cur_temp = await curr_temp_async(city)
    is_normal = is_temperature_normal(city, season, cur_temp, df_grouped)
    return cur_temp, is_normal

# await check_temperature_async("New York", "winter", grouped_df)

Далее проверим скорость работу реализованных функций в синхронном и асинхронном режиме

In [8]:
def main_sync(cities, season, data):
    df_grouped = data.groupby(['city', 'season'])['temperature'].agg(['mean', 'std']).reset_index()
    results = {}
    for city in cities:
        try:
            current_temp, is_normal = check_temperature_sync(city, season, df_grouped)
            results[city] = {"Температура": current_temp, "Нормальность температуры": is_normal}
        except Exception as e:
            results[city] = {"Error ": str(e)}

    return results

# main_sync(["New York"], "winter",df)

In [9]:
async def main_async(cities, season, data):
    df_grouped = data.groupby(['city', 'season'])['temperature'].agg(['mean', 'std']).reset_index()
    results = {}
    tasks = []
    for city in cities:
        tasks.append(check_temperature_async(city, season, df_grouped))
    responses = await asyncio.gather(*tasks, return_exceptions=True)
    for city, response in zip(cities, responses):
        if isinstance(response, Exception):
            results[city] = {"Error ": str(response)}
        else:
            current_temp, is_normal = response
            results[city] = {"Температура": current_temp,"Нормальность температуры": is_normal}
    return results


# await main_async(["New York","Berlin","Cairo","Abc"], "winter",df)

In [10]:
cities = ["London", "Moscow", "Paris", "Mumbai"]

In [11]:
start_time = time.time()
sync_results = main_sync(cities, "winter", df)
time_sync = round(time.time() - start_time,4)


print("Синхронное выполнение")
print(f"Результат : {sync_results}")
print(f"Заняло времени: {time_sync} секунд")

Синхронное выполнение
Результат : {'London': {'Температура': 2.47, 'Нормальность температуры': True}, 'Moscow': {'Температура': -0.55, 'Нормальность температуры': True}, 'Paris': {'Температура': 4.53, 'Нормальность температуры': True}, 'Mumbai': {'Температура': 25.99, 'Нормальность температуры': True}}
Заняло времени: 3.9088 секунд


In [12]:
start_time = time.time()
async_results = asyncio.run(main_async(cities, "winter", df))
time_async = round(time.time() - start_time,4)

print("Асинхронное выполнение")
print(f"Результат {async_results}")
print(f"Заняло времени: {time_async} секунд")

Асинхронное выполнение
Результат {'London': {'Температура': 2.47, 'Нормальность температуры': True}, 'Moscow': {'Температура': -0.55, 'Нормальность температуры': True}, 'Paris': {'Температура': 4.53, 'Нормальность температуры': True}, 'Mumbai': {'Температура': 25.99, 'Нормальность температуры': True}}
Заняло времени: 0.8983 секунд


Как видим, в асинхронном режиме функицонал работает быстрее, так как асинхронность как раз необходима для оптимизации такого типа задач